<img src="./files/allfed_logo.png">

# Agricultural Shipping Route Analysis

Developing effective approaches to feeding everyone in the event of a global catastrophe involves understanding how different agricultural products can be shipped within and between countries.
In this notebook I'll utilise code developed by ALLFED to build up a data representation of agricultural shipping routes, and use that model to perform analysis on the movement of livestock and agricultural residues.

The initial test data we will be using for this exercise is in Tasmania, Australia, and consists of rail data, road data, the location of livestock, and the location of cropland. We'll be using datasets which are available globally, meaning we can readily scale up the present analysis.

Shipping route data | Livestock data | Cropland data
- | - | -
<img src="./files/tas_routes.png"> | <img src="./files/tas_livestock.png"> | <img src="./files/tas_crops.png">





The code used here was developed for the explicit purpose of doing global spatial analysis, and can be found at https://github.com/allfed/allfed-spatial 

### Setting up

In [3]:
# Allow access to Python imports from higher level folder
import sys
sys.path.append('..')

# Define directory locations
TEST_DIR = '../test/'
IN_SHAPE_DIR = '../test/in_shape/'
GRAPH_DIR = '../test/graph'

###  1. Preprocess route data

First we'll load in our route data and make sure it's all geometrically connected. It's important that routes are properly connected as we use these connections to infer traversible routes. 

Some source data describing (for example) roads will appear connected, but will actually be disconnected when zoomed in. To address this, we'll "snap" together line segments which are close to each other.

Road network | Zoomed in | Fixed via 'snapping'
- | - | -
<img src="./files/route_network_highlighted.png"> | <img src="./files/route_network_disconnect.png"> | <img src="./files/route_network_snapped.png">

In [4]:
import fiona
from shapely.geometry import LineString, shape

# ALLFED libs
from geometry.snap import snap_linestrings
from geometry.io import write_shape
from geometry.project import project_to_utm, project_from_utm

In [7]:
# Load road shapefile (just look at roads for now)
ROAD_PATH = TEST_DIR + 'ne_10m_roads_test.shp'
roadF = fiona.open(ROAD_PATH)
road_geoms = [LineString(shape(f['geometry'])) for f in roadF]
road_data = [f['properties'] for f in roadF]

# Project to UTM coordinate system
road_geoms = project_to_utm(road_geoms, epsg='28355', zone=55)

# Create new snapped LineString collection
SNAPPED_FILE_NAME = 'snapped.shp'
snapped = snap_linestrings(500, road_geoms) # snap line endpoints which are within 500 metres of each other

# Write test output from this stage
test_output = project_from_utm(snapped, epsg='28355', zone=55)
write_shape(test_output, road_data, roadF.schema, TEST_DIR + SNAPPED_FILE_NAME)

### 2. Preprocess livestock and crop data

We now want to connect our livestock and crop data to our route network. The livestock and crop data is in a "raster" format, consisting of pixels with assigned values representing livestock numbers, which we want to preserve. To connect to our network, we'll first convert each pixel into a spatial point at the pixel centroid, with data attributes indicating the number of livestock at that average location. We'll then connect each point to the closest available route.

Initial data | Converted to points | Connected to route network
- | - | -
<img src="./files/tas_livestock.png"> | <img src="./files/tas_livestock_points.png"> | <img src="./files/tas_livestock_network.png">

In [10]:
from collections import OrderedDict

# ALLFED libs
from raster.conversions import raster_to_points
from geometry.line import split_line_by_distance, join_points_to_lines

In [11]:
# Load livestock raster (just look at livestock for now)
LIVESTOCK_PATH = TEST_DIR + 'csiro_bvmeat_feed_00_test.tif'
point_geoms, point_data = raster_to_points(LIVESTOCK_PATH)
point_geoms = project_to_utm(point_geoms, epsg='28355', zone=55)

# Split out network every 10km to provide regularly spaced attachment points
split_lines = []
for s in snapped:
    split_lines.extend(split_line_by_distance(s, 10000))
    
# Join our livestock points to the network
joins = join_points_to_lines(point_geoms, split_lines)


# Write test output from this stage
schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {'value': 'float:16'}
})
test_output = project_from_utm(joins, epsg='28355', zone=55)
write_shape(test_output, point_data, schema, TEST_DIR + 'join_test.shp')

schema = OrderedDict({
    'geometry': 'Point',
    'properties': {'value': 'float:16'}
})
test_output = project_from_utm(point_geoms, epsg='28355', zone=55)
write_shape(test_output, point_data, schema, TEST_DIR + 'point_test.shp')

### 3. Convert to graph and run analysis

We'll now convert our processed data into a [graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)) - a mathematical structure consisting of nodes and edges. This structure allows us to easily perform analysis on our data.

In [13]:
import networkx as nx

In [15]:
# Write our preprocessed data to our in shape folder
cleaned_livestock_edges = project_from_utm(joins, epsg='28355', zone=55)
schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {'value': 'float:16'}
})
write_shape(cleaned_livestock_edges, point_data, schema, IN_SHAPE_DIR + 'livestock_edges.shp')

cleaned_route_edges = project_from_utm(split_lines, epsg='28355', zone=55)
schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {}
})
write_shape(cleaned_route_edges, [{} for b in cleaned_route_edges], schema, IN_SHAPE_DIR + 'road_edges.shp')

In [16]:
# Read back in as a graph, then write again to inspect
Groad = nx.read_shp(IN_SHAPE_DIR)
nx.write_shp(Groad, GRAPH_DIR)

In [18]:
# Test out shortest path
try:
    n = nx.shortest_path_length(Groad.reverse(copy=False),(148.24732818407236, -41.513903951489986))
    print(n)
except nx.NetworkXNoPath:
    print('No path')

{(148.24732818407236, -41.513903951489986): 0, (148.2466090633311, -41.514623235975165): 1, (148.18113577579263, -41.5798955910601): 2, (148.15041605625387, -41.52116266899999): 2, (148.06210256350963, -41.58468200860842): 3, (147.90041605623847, -41.437829335666656): 3, (148.15041605625387, -41.60449600233333): 3, (147.96502907958828, -41.63886243002555): 4, (147.98374938957693, -41.43782933566666): 4, (148.0670827229154, -41.43782933566666): 4, (147.90041605623847, -41.52116266899999): 4, (147.98374938957693, -41.521162669): 4, (148.0670827229154, -41.52116266899999): 4, (148.0670827229154, -41.60449600233332): 4, (148.0670827229154, -41.687829335666656): 4, (148.15041605625387, -41.687829335666656): 4, (147.87695583114817, -41.70018201952903): 5, (147.73374938956155, -41.52116266899999): 5, (147.8170827229, -41.521162669): 5, (147.8170827229, -41.60449600233332): 5, (147.90041605623847, -41.60449600233333): 5, (147.98374938957693, -41.68782933566665): 5, (147.98374938957693, -41.771